# BIZ 프로젝트 : 부실기업 예측

## Step1 : 데이터 불러오기

외부감사를 받아야 하는 법인을 의미하는 외부감사기업을 대상으로    
20-24년의 기간의 재무비율 115개 항목에 대해서 ValueSearch를 이용하여 데이터를 수집하였다.  

https://www.nicevse.com/vse/main.html

### 필요라이브러리 불러오기

In [16]:
import pandas as pd
import numpy as np

### 데이터 불러오기

In [17]:
# # Excel 파일 읽기
# data = pd.read_excel("../../project/data/financial_data.xlsx", header=1, sheet_name='Sheet1')

# # CSV 파일로 저장
# data.to_csv("../../project/data/financial_data.csv", index=False, encoding='utf-8-sig')

xlsx 파일로 불러오는 경우 오랜시간이 걸려 csv 파일로 변환하여 사용

In [18]:
RANDOM_STATE = 110

data = pd.read_csv("../../project/data/financial_data.csv", encoding='utf-8-sig', low_memory=False)

In [19]:
len(data)

39861

외부감사기업으로 약 4만개의 기업의 데이터를 확보하였다.

In [20]:
data.columns

Index(['업체코드', '종목코드', '종목명', '_0', '_0.1', '_0.2', '_0.3', '_0.4',
       '2020/Annual\n191000.<성장성에 관한 지표>', '2021/Annual\n191000.<성장성에 관한 지표>',
       ...
       '2020/Annual\n196050.조세공과(구성비)', '2021/Annual\n196050.조세공과(구성비)',
       '2022/Annual\n196050.조세공과(구성비)', '2023/Annual\n196050.조세공과(구성비)',
       '2024/Annual\n196050.조세공과(구성비)', '2020/Annual\n196060.감가상각비(구성비)',
       '2021/Annual\n196060.감가상각비(구성비)', '2022/Annual\n196060.감가상각비(구성비)',
       '2023/Annual\n196060.감가상각비(구성비)', '2024/Annual\n196060.감가상각비(구성비)'],
      dtype='object', length=578)

In [21]:
# 확인할 변수 목록
variables_to_check = ['_0', '_0.1', '_0.2', '_0.3', '_0.4']

# 각 변수의 고유 값 확인
for column in variables_to_check:
    if column in data.columns:
        unique_values = data[column].unique()
        print(f"'{column}' 변수의 고유 값: {unique_values}")
    else:
        print(f"'{column}' 변수는 데이터프레임에 존재하지 않습니다.")

'_0' 변수의 고유 값: [nan]
'_0.1' 변수의 고유 값: [nan]
'_0.2' 변수의 고유 값: [nan]
'_0.3' 변수의 고유 값: [nan]
'_0.4' 변수의 고유 값: [nan]


In [22]:
# 드랍할 변수 목록
variables_to_drop = ['_0', '_0.1', '_0.2', '_0.3', '_0.4']

# 변수 드랍
data = data.drop(columns=variables_to_drop)

데이터를 수집하는 과정에서 불필요하다고 생각이 들은 변수들에 대해서 결측값을 확인하였고    
['_0', '_0.1', '_0.2', '_0.3', '_0.4'] 5개의 변수 모두 결측으로 존재하여 제거해 주었다.  

In [23]:
data.columns

Index(['업체코드', '종목코드', '종목명', '2020/Annual\n191000.<성장성에 관한 지표>',
       '2021/Annual\n191000.<성장성에 관한 지표>', '2022/Annual\n191000.<성장성에 관한 지표>',
       '2023/Annual\n191000.<성장성에 관한 지표>', '2024/Annual\n191000.<성장성에 관한 지표>',
       '2020/Annual\n191010.총자산증가율', '2021/Annual\n191010.총자산증가율',
       ...
       '2020/Annual\n196050.조세공과(구성비)', '2021/Annual\n196050.조세공과(구성비)',
       '2022/Annual\n196050.조세공과(구성비)', '2023/Annual\n196050.조세공과(구성비)',
       '2024/Annual\n196050.조세공과(구성비)', '2020/Annual\n196060.감가상각비(구성비)',
       '2021/Annual\n196060.감가상각비(구성비)', '2022/Annual\n196060.감가상각비(구성비)',
       '2023/Annual\n196060.감가상각비(구성비)', '2024/Annual\n196060.감가상각비(구성비)'],
      dtype='object', length=573)

기업의 인덱스와 관련된 변수 ['업체코드', '종목코드', '종목명']를 제외하면 변수는 총 570개  
570 -> 5(연도) * 114(재무비율) 으로 재무비율 변수로 총 114개  

In [24]:
# 열 이름 변환 함수
def transform_column_name(col_name):
    if col_name in ['업체코드', '종목코드', '종목명']:
        return col_name
    parts = col_name.split('\n')
    if len(parts) == 2:
        year, desc = parts
        desc = desc.split('.')[1]  # 변수 키워드 부분만 추출
        return f"{year.split('/')[0]}/{desc}"
    return col_name

# 열 이름 변환 적용
data.columns = [transform_column_name(col) for col in data.columns]

# 변환된 열 이름 출력
print(data.columns)

Index(['업체코드', '종목코드', '종목명', '2020/<성장성에 관한 지표>', '2021/<성장성에 관한 지표>',
       '2022/<성장성에 관한 지표>', '2023/<성장성에 관한 지표>', '2024/<성장성에 관한 지표>',
       '2020/총자산증가율', '2021/총자산증가율',
       ...
       '2020/조세공과(구성비)', '2021/조세공과(구성비)', '2022/조세공과(구성비)', '2023/조세공과(구성비)',
       '2024/조세공과(구성비)', '2020/감가상각비(구성비)', '2021/감가상각비(구성비)',
       '2022/감가상각비(구성비)', '2023/감가상각비(구성비)', '2024/감가상각비(구성비)'],
      dtype='object', length=573)


변수명의 주요 키워드만을 이용하여 새로운 변수명으로 변환   
ex) 2020/Annual\n196050.조세공과(구성비) -> 2020/조세공과(구성비)   

In [25]:
# 열 이름 중복 확인 함수
def check_duplicate_columns(df):
    duplicates = df.columns[df.columns.duplicated()]
    if len(duplicates) > 0:
        print("중복되는 열 이름:", duplicates)
    else:
        print("중복되는 열 이름이 없습니다.")

# 중복 확인 함수 호출
check_duplicate_columns(data)

중복되는 열 이름이 없습니다.


변환된 변수명간에 중복되는 경우를 확인하였고 없음을 확인하였다.

다음으로 재무비율 변수들중에서 모든행의 값이 결측인 경우 제거

In [26]:
import pandas as pd

# 값이 하나도 없는 열 확인
empty_cols = data.columns[data.isnull().all()]

# 값이 하나도 없는 열 출력
print("값이 하나도 없는 열:")
print(empty_cols)

# 값이 하나도 없는 열 드롭
data = data.drop(columns=empty_cols)

# 드롭된 후 데이터프레임 확인
print(f"\n드롭된 후 데이터프레임의 열 수: {data.shape[1]}")

# empty_cols의 열 이름에서 / 뒤에 있는 이름만 추출하여 중복 없이 출력
unique_names = set()
print("\n값이 하나도 없는 변수 목록:")
for col in empty_cols:
    if '/' in col:
        name = col.split('/')[-1]
    else:
        name = col
    if name not in unique_names:
        unique_names.add(name)
        print(name)

값이 하나도 없는 열:
Index(['2020/<성장성에 관한 지표>', '2021/<성장성에 관한 지표>', '2022/<성장성에 관한 지표>',
       '2023/<성장성에 관한 지표>', '2024/<성장성에 관한 지표>', '2020/<손익의 관계비율>',
       '2021/<손익의 관계비율>', '2022/<손익의 관계비율>', '2023/<손익의 관계비율>',
       '2024/<손익의 관계비율>', '2020/<자산자본의 관계비율>', '2021/<자산자본의 관계비율>',
       '2022/<자산자본의 관계비율>', '2023/<자산자본의 관계비율>', '2024/<자산자본의 관계비율>',
       '2020/<자산자본의 회전율>', '2021/<자산자본의 회전율>', '2022/<자산자본의 회전율>',
       '2023/<자산자본의 회전율>', '2024/<자산자본의 회전율>', '2020/<생산성에 관한지표>',
       '2021/<생산성에 관한지표>', '2022/<생산성에 관한지표>', '2023/<생산성에 관한지표>',
       '2024/<생산성에 관한지표>', '2020/<부가가치의 구성>', '2021/<부가가치의 구성>',
       '2022/<부가가치의 구성>', '2023/<부가가치의 구성>', '2024/<부가가치의 구성>'],
      dtype='object')

드롭된 후 데이터프레임의 열 수: 543

값이 하나도 없는 변수 목록:
<성장성에 관한 지표>
<손익의 관계비율>
<자산자본의 관계비율>
<자산자본의 회전율>
<생산성에 관한지표>
<부가가치의 구성>


<성장성에 관한 지표>, <손익의 관계비율>, <자산자본의 관계비율> ,   
<자산자본의 회전율> , <생산성에 관한지표>, <부가가치의 구성>    
6가지 재무비율에 대해서 제거-> 30개(= 5(연도) * 6(재무비율))의 변수 제거    

분석하는 현재 연도는 2024년으로 재무비율의 값이 발표가 아직 안된경우를 고려하여  
20~23년도 기준으로 고려하였고 해당 기간동안 모든변수의 값이 결측으로 존재하는 경우를 확인하여  
이후 예측에 있어 불필요한 데이터에 대해서 제거하고자 하였습니다.

In [27]:
# 특정단어를 포함하는 열 이름 필터링
keywords = ['2020', '2021', '2022', '2023']
filter_cols = [col for keyword in keywords for col in data.filter(like=keyword).columns]

# 모든 변수가 결측치인 행 확인
missing_all = data[filter_cols].isnull().all(axis=1)

# 결측치인 행의 수
num_missing_all = missing_all.sum()
print(f"\n{keywords} 기간 동안 모든 변수가 결측치인 행의 수: {num_missing_all}")

# 결측치인 행의 업체코드 리스트 추출 -> 이후 행을 제거할 때 사용
missing_companies = data.loc[missing_all, '업체코드'].tolist()
# print("\n모든 변수가 결측치인 행의 업체코드 리스트:")
# print(missing_companies)

# 해당하는 행의 종목명 값들 출력 -> 종목명을 통해 어떤 종목인지 확인용도
missing_items = data.loc[missing_all, '종목명'].unique()

# 모든 값을 출력하도록 설정
# np.set_printoptions(threshold=np.inf)

print("\n모든 변수가 결측치인 행의 종목명 값들:")
print(missing_items)


['2020', '2021', '2022', '2023'] 기간 동안 모든 변수가 결측치인 행의 수: 1222

모든 변수가 결측치인 행의 종목명 값들:
['(주)신한은행' '(주)우리은행' '메리츠화재해상보험(주)' ... '한화모멘텀(주)'
 '이지스제420호전문투자형사모부동산투자회사' '(주)태영에이엠씨대부']


해당하는 기업의 경우 금융업에 해당하는 기업으로 금융업 관련 기업을 제거함으로서  
향후 부실기업에 예측에 유의미함을 위해 제거하였다.  

In [28]:
# 제거된 후 데이터프레임 확인
print(f"\n제거하기 전 데이터프레임의 행 수: {len(data)}")

# 해당하는 기업들을 제거 -> 업체코드를 기준으로 제거
data = data[~data['업체코드'].isin(missing_companies)]

print(f"\n제거하는 행 수: {num_missing_all}")

# 제거된 후 데이터프레임 확인
print(f"\n제거된 후 데이터프레임의 행 수: {len(data)}")


제거하기 전 데이터프레임의 행 수: 39861

제거하는 행 수: 1222

제거된 후 데이터프레임의 행 수: 38639


기업의 갯수 38639개

In [29]:
# 특정단어를 포함하는 열 이름 필터링
keyword = '2021'
filter_col = data.filter(like=keyword).columns

# print(f"< {keyword} 포함 변수 >")
# for col in filter_col:
#     print(col)

print(len(filter_col))   

108


연도구분없이 순수 재무비율의 종류만 108개  

### 데이터 저장

In [30]:
# CSV 파일로 저장
data.to_csv("../../project/data/clean_financial_data.csv", index=False, encoding='utf-8-sig')

.